<a href="https://colab.research.google.com/github/LuizGPassos/ted-rfv/blob/main/T%26D_RFV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio

- Explorar distribuições de R, F, V
- Identificar outliers e padrões
- Definir critérios claros de segmentação
- Validar se segmentos fazem sentido


# Imports e Configs

In [1]:
!wget -O customer_dataset_ted.csv "https://gist.githubusercontent.com/LuizGPassos/1ab4e770bc9cc495fc990d2b529a170e/raw/8da68f748d29adbedef5210a7244c9f5ca9de5e6/customer_dataset_ted.csv"

--2025-08-11 17:25:12--  https://gist.githubusercontent.com/LuizGPassos/1ab4e770bc9cc495fc990d2b529a170e/raw/8da68f748d29adbedef5210a7244c9f5ca9de5e6/customer_dataset_ted.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2570 (2.5K) [text/plain]
Saving to: ‘customer_dataset_ted.csv’

customer_dataset_te 100%[===================>]   2.51K  --.-KB/s    in 0s      

2025-08-11 17:25:12 (14.8 MB/s) - ‘customer_dataset_ted.csv’ saved [2570/2570]



In [2]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.getOrCreate()

# Tabelas e Variáveis

In [4]:
df = spark.read.csv("customer_dataset_ted.csv",header=True,inferSchema=True)
df.show()

+-----------+-----------------+-------------------+-------------------+------------+-----------+------------------+-------------+---------------+------------------------+------------------------+
|customer_id|favorite_category|acquisition_channel|first_purchase_date|total_orders|total_spent|last_purchase_date|returns_count|avg_order_value|days_since_last_purchase|customer_lifetime_months|
+-----------+-----------------+-------------------+-------------------+------------+-----------+------------------+-------------+---------------+------------------------+------------------------+
|   CUST0001|          Tablets|             Google|         2022-11-16|          10|      17302|        2024-03-25|            0|           1730|                     124|                      20|
|   CUST0002|            Games|             Google|         2023-04-18|          21|      18160|        2024-07-21|            0|            865|                       6|                      15|
|   CUST0003|       

In [5]:
df = (
    df
    .withColumn("first_purchase_date", to_date("first_purchase_date"))
    .withColumn("last_purchase_date", to_date("last_purchase_date"))
    .withColumn("total_orders", col("total_orders").cast("int"))
    .withColumn("total_spent", col("total_spent").cast("double"))
    .withColumn("avg_order_value", col("avg_order_value").cast("double"))
    .withColumn("days_since_last_purchase", col("days_since_last_purchase").cast("int"))
    .withColumn("customer_lifetime_months", col("customer_lifetime_months").cast("int"))
    .withColumn("returns_count", col("returns_count").cast("int"))
)
df.describe()

DataFrame[summary: string, customer_id: string, favorite_category: string, acquisition_channel: string, total_orders: string, total_spent: string, returns_count: string, avg_order_value: string, days_since_last_purchase: string, customer_lifetime_months: string]

# Querys

In [6]:
rfv = (
    df
    .select(
        "customer_id","favorite_category","acquisition_channel",
        "first_purchase_date","last_purchase_date","customer_lifetime_months","returns_count",
        col("days_since_last_purchase").alias("R"),
        col("total_orders").alias("F"),
        col("total_spent").alias("V"),
        col("avg_order_value").alias("AOV")
    )
)

rfv.show()

+-----------+-----------------+-------------------+-------------------+------------------+------------------------+-------------+---+---+-------+------+
|customer_id|favorite_category|acquisition_channel|first_purchase_date|last_purchase_date|customer_lifetime_months|returns_count|  R|  F|      V|   AOV|
+-----------+-----------------+-------------------+-------------------+------------------+------------------------+-------------+---+---+-------+------+
|   CUST0001|          Tablets|             Google|         2022-11-16|        2024-03-25|                      20|            0|124| 10|17302.0|1730.0|
|   CUST0002|            Games|             Google|         2023-04-18|        2024-07-21|                      15|            0|  6| 21|18160.0| 865.0|
|   CUST0003|          Tablets|           Orgânico|         2023-01-10|        2024-04-30|                      18|            0| 88| 16| 7309.0| 457.0|
|   CUST0004|            Audio|           Facebook|         2022-08-11|        202

In [7]:
summary_exprs = {
    "R": ["count","mean","percentile_approx"],
    "F": ["mean","percentile_approx"],
    "V": ["mean","percentile_approx"],
    "AOV": ["mean","percentile_approx"]
}
percentis = [0.25, 0.5, 0.75, 0.9]

stats = (
    rfv
    .agg(
        count("*").alias("n_clientes"),
        mean("R").alias("R_media"),
        expr(f"percentile_approx(R, array({','.join(map(str,percentis))}))").alias("R_pct"),
        mean("F").alias("F_media"),
        expr(f"percentile_approx(F, array({','.join(map(str,percentis))}))").alias("F_pct"),
        mean("V").alias("V_media"),
        expr(f"percentile_approx(V, array({','.join(map(str,percentis))}))").alias("V_pct"),
        mean("AOV").alias("AOV_media"),
        expr(f"percentile_approx(AOV, array({','.join(map(str,percentis))}))").alias("AOV_pct"),
    )
)

stats.show()

+----------+-------+------------------+-----------------+--------------+------------------+--------------------+-----------------+--------------------+
|n_clientes|R_media|             R_pct|          F_media|         F_pct|           V_media|               V_pct|        AOV_media|             AOV_pct|
+----------+-------+------------------+-----------------+--------------+------------------+--------------------+-----------------+--------------------+
|        35|   92.2|[10, 53, 164, 242]|10.82857142857143|[5, 8, 16, 23]|7965.5142857142855|[2945.0, 6432.0, ...|766.8571428571429|[551.0, 759.0, 88...|
+----------+-------+------------------+-----------------+--------------+------------------+--------------------+-----------------+--------------------+



In [8]:
def iqr_bounds(df, col):
    q = df.approxQuantile(col, [0.25, 0.75], 0.01)
    Q1, Q3 = q[0], q[1]
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    return lower, upper

bounds = {}
for c in ["R","F","V","AOV"]:
    bounds[c] = iqr_bounds(rfv, c)

rfv_out = (
    rfv
    .withColumn("out_R",  (col("R")  < lit(bounds["R"][0]))  | (col("R")  > lit(bounds["R"][1])))
    .withColumn("out_F",  (col("F")  < lit(bounds["F"][0]))  | (col("F")  > lit(bounds["F"][1])))
    .withColumn("out_V",  (col("V")  < lit(bounds["V"][0]))  | (col("V")  > lit(bounds["V"][1])))
    .withColumn("out_AOV",(col("AOV")< lit(bounds["AOV"][0]))| (col("AOV")> lit(bounds["AOV"][1])))
)

rfv_out.show()

+-----------+-----------------+-------------------+-------------------+------------------+------------------------+-------------+---+---+-------+------+-----+-----+-----+-------+
|customer_id|favorite_category|acquisition_channel|first_purchase_date|last_purchase_date|customer_lifetime_months|returns_count|  R|  F|      V|   AOV|out_R|out_F|out_V|out_AOV|
+-----------+-----------------+-------------------+-------------------+------------------+------------------------+-------------+---+---+-------+------+-----+-----+-----+-------+
|   CUST0001|          Tablets|             Google|         2022-11-16|        2024-03-25|                      20|            0|124| 10|17302.0|1730.0|false|false|false|   true|
|   CUST0002|            Games|             Google|         2023-04-18|        2024-07-21|                      15|            0|  6| 21|18160.0| 865.0|false|false|false|  false|
|   CUST0003|          Tablets|           Orgânico|         2023-01-10|        2024-04-30|               

In [9]:

qs = [0.2,0.4,0.6,0.8]
qR = rfv.approxQuantile("R", qs, 0.01)
qF = rfv.approxQuantile("F", qs, 0.01)
qV = rfv.approxQuantile("V", qs, 0.01)

def score_from_quantiles(col, q, reverse=False):
    conds = [
        (col <= q[0], 1),
        (col <= q[1], 2),
        (col <= q[2], 3),
        (col <= q[3], 4),
    ]

    def expr():
        e = when(conds[0][0], 1).when(conds[1][0], 2).when(conds[2][0], 3).when(conds[3][0], 4).otherwise(5)
        return (6 - e) if reverse else e
    return expr()

rfv_scored = (
    rfv_out
    .withColumn("R_score", score_from_quantiles(col("R"), qR, reverse=True))
    .withColumn("F_score", score_from_quantiles(col("F"), qF, reverse=False))
    .withColumn("V_score", score_from_quantiles(col("V"), qV, reverse=False))
    .withColumn("RFV_score", col("R_score")+col("F_score")+col("V_score"))
)
rfv_scored.show()

+-----------+-----------------+-------------------+-------------------+------------------+------------------------+-------------+---+---+-------+------+-----+-----+-----+-------+-------+-------+-------+---------+
|customer_id|favorite_category|acquisition_channel|first_purchase_date|last_purchase_date|customer_lifetime_months|returns_count|  R|  F|      V|   AOV|out_R|out_F|out_V|out_AOV|R_score|F_score|V_score|RFV_score|
+-----------+-----------------+-------------------+-------------------+------------------+------------------------+-------------+---+---+-------+------+-----+-----+-----+-------+-------+-------+-------+---------+
|   CUST0001|          Tablets|             Google|         2022-11-16|        2024-03-25|                      20|            0|124| 10|17302.0|1730.0|false|false|false|   true|      2|      3|      5|       10|
|   CUST0002|            Games|             Google|         2023-04-18|        2024-07-21|                      15|            0|  6| 21|18160.0| 86

In [10]:
rfv_segments = (
    rfv_scored
    .withColumn(
        "segment",
          when( (col("R_score")>=4) & (col("F_score")>=4) & (col("V_score")>=4), "Champions")
         .when( (col("R_score")>=4) & (col("F_score")>=3), "Leais")
         .when( (col("R_score")<=2) & (col("F_score")>=4) & (col("V_score")>=3), "Em Risco (VIP sumido)")
         .when( (col("R_score")>=3) & (col("V_score")>=4) & (col("F_score")<=2),"Grandes Gastos Ocas.")
         .when( (col("R_score")>=3) & (col("F_score")<=2) & (col("V_score")<=2),"Novos / Potenciais")
         .otherwise("Morno")
    )
)

seg_sizes = rfv_segments.groupBy("segment").agg(
    count("*").alias("n"),
    mean("R").alias("R_med"),
    mean("F").alias("F_med"),
    mean("V").alias("V_med"),
    mean("AOV").alias("AOV_med"),
    mean("returns_count").alias("devol_med")
).orderBy(desc("n"))

seg_sizes.show()

+--------------------+---+-----------------+------------------+------------------+-----------------+-------------------+
|             segment|  n|            R_med|             F_med|             V_med|          AOV_med|          devol_med|
+--------------------+---+-----------------+------------------+------------------+-----------------+-------------------+
|               Morno| 16|           170.25|             7.875|         5883.0625|           763.75|              1.125|
|           Champions| 11|6.636363636363637|20.272727272727273|15081.636363636364|770.4545454545455| 0.7272727272727273|
|  Novos / Potenciais|  7|             38.0|2.5714285714285716|            1762.0|801.2857142857143|0.42857142857142855|
|Em Risco (VIP sum...|  1|            164.0|              12.0|            6432.0|            536.0|                2.0|
+--------------------+---+-----------------+------------------+------------------+-----------------+-------------------+



In [11]:
rfv_buckets = (
    rfv_segments
    .withColumn("R_bin", when(col("R")<=qR[0],"R1").when(col("R")<=qR[1],"R2").when(col("R")<=qR[2],"R3").when(col("R")<=qR[3],"R4").otherwise("R5"))
    .withColumn("F_bin", when(col("F")<=qF[0],"F1").when(col("F")<=qF[1],"F2").when(col("F")<=qF[2],"F3").when(col("F")<=qF[3],"F4").otherwise("F5"))
    .withColumn("V_bin", when(col("V")<=qV[0],"V1").when(col("V")<=qV[1],"V2").when(col("V")<=qV[2],"V3").when(col("V")<=qV[3],"V4").otherwise("V5"))
)

rxv = (
    rfv_buckets.groupBy("R_bin")
    .pivot("V_bin")
    .agg(mean("V").alias("V_med"))
    .orderBy("R_bin")
)

canal_seg = (
    rfv_buckets.groupBy("acquisition_channel","segment")
    .agg(count("*").alias("n"), mean("V").alias("V_med"))
    .orderBy("acquisition_channel","segment")
)

canal_seg.show()

+-------------------+--------------------+---+-----------------+
|acquisition_channel|             segment|  n|            V_med|
+-------------------+--------------------+---+-----------------+
|              Email|               Morno|  5|           5433.0|
|              Email|  Novos / Potenciais|  2|           3122.5|
|           Facebook|Em Risco (VIP sum...|  1|           6432.0|
|           Facebook|               Morno|  4|          4590.75|
|           Facebook|  Novos / Potenciais|  2|            706.0|
|             Google|           Champions|  5|          14017.8|
|             Google|               Morno|  3|7612.333333333333|
|          Indicação|           Champions|  5|          15636.6|
|          Indicação|  Novos / Potenciais|  1|           2100.0|
|           Orgânico|           Champions|  1|          17626.0|
|           Orgânico|               Morno|  4|           6441.0|
|           Orgânico|  Novos / Potenciais|  2|           1288.5|
+-------------------+----

In [12]:
rfv_cohort = (
    rfv_buckets
    .withColumn("cohort_month", date_format(trunc("first_purchase_date","MM"), "yyyy-MM"))
    .groupBy("cohort_month")
    .agg(
        count("*").alias("n_clientes"),
        mean("F").alias("F_med"),
        mean("V").alias("V_med"),
        mean("R").alias("R_med")
    )
    .orderBy("cohort_month")
)

rfv_cohort.show()


+------------+----------+-----+------------------+------------------+
|cohort_month|n_clientes|F_med|             V_med|             R_med|
+------------+----------+-----+------------------+------------------+
|     2022-07|         2|  4.5|            3921.0|             270.5|
|     2022-08|         3|  7.0| 5859.333333333333|             182.0|
|     2022-09|         3| 19.0|           12728.0|30.666666666666668|
|     2022-10|         1| 11.0|            8765.0|              18.0|
|     2022-11|         3| 12.0|12989.333333333334|              60.0|
|     2022-12|         2| 14.5|           12199.5|               8.0|
|     2023-01|         4| 11.5|           7268.25|            118.75|
|     2023-02|         3| 11.0|            9495.0|133.33333333333334|
|     2023-03|         1|  3.0|            2789.0|              69.0|
|     2023-04|         3| 19.0|14489.333333333334| 69.66666666666667|
|     2023-05|         2| 19.0|           10541.0|              27.0|
|     2023-10|      